# spotidata Interactive
Uses the spotidata command-line toolsuite to fetch songs from your saved music library, saved albums, and playlists, output/analyze/graph the data, and 

In [ ]:
import sys
sys.path.append('../')

import api.spotify as spotify
import api.utils as utils

from api.spotify import FeatureType, FeatureFilter

import clt.input_tools as clinput
import clt.spotidata as spotidata

First, we need to load the application data and username, and login.

In [ ]:
username, scope = spotidata.boot('../config.yml')
with spotify.login(username, scope) as sp_login:
    sp = sp_login

Now, you'll need to choose where you want your music from.

In [ ]:
use_saved_tracks = clinput.input_boolean("Do you want me to analyze your saved songs?")
print()
print("Saved Albums:")
queried_albums = clinput.input_sublist("Which albums would you like to include in the analysis?", sp.get_saved_albums())
print()
print("Playlists:")
queried_playlists = clinput.input_sublist("Which playlists would you like to include in the analysis?", sp.get_playlists())

print("="*12)
print("including songs from:")
if use_saved_tracks:
    print("  - your Saved Music library")
if len(queried_albums) != 0:
    print("  - the following albums: {}".format(utils.get_english_list(queried_albums)))
if len(queried_playlists) != 0:
    print("  - the following playlists: {}".format(utils.get_english_list(queried_playlists)))

Let's pull all that music and grab the features.

In [ ]:
print("Fetching songs...")
tracks = sp.get_tracks(use_saved_tracks, queried_albums, queried_playlists)
print("Getting features...")
for track in tracks:
    track.load_features(sp)
print("Done.")

Now, we need to define the predicate that is used to filter the songs.

In [ ]:
# These filters attempt to make a lower-energy playlist.
filters = [
    FeatureFilter(FeatureType.ENERGY,       min_val=   0.0, max_val=   0.7),
    FeatureFilter(FeatureType.TEMPO,        min_val=   0.0, max_val= 130.0),
    FeatureFilter(FeatureType.LOUDNESS,     min_val= -60.0, max_val=  -6.0)
]

predicate = lambda track : track.satisfies_all(filters)

All that's left to do is make the playlist!

In [ ]:
constituent_tracks = sp.get_tracks_that_satisfy_predicate(tracks, predicate)
sp.create_playlist("chill", constituent_tracks)